# Textklassifikation mit DraCor

[DraCor](https://dracor.org/) ist ein Korpus an Dramen, die mit Hilfe einer umfangreichen [API](https://dracor.org/doc/api) zur Verfügung gestellt werden. In diesem Notebook testen wir, inwiefern sich die Autor:innen der Dramen anhand ihrer Texte identifizieren lassen. Dies ist eine typisches Anwendung der [Stilometrie](https://en.wikipedia.org/wiki/Stylometry).

## Korpuserstellung

Die ersten beiden Funktionen dienen dazu, ein Korpus an Dramen von DraCor herunterzuladen:

In [ ]:
from urllib import request
import json 

dracor_api = "https://dracor.org/api"                # API-Endpunkt für DraCor


def get_dracor(corpus, play=None):
    """Lädt entweder Metadaten zum Korpus oder den Text des Stücks."""
    url = dracor_api + "/corpora/" + corpus          # Basis-URL
    if play is not None:                             # Stück gewünscht?
        url = url + "/play/" + play + "/spoken-text" # URL für Text des Stückes
    with request.urlopen(url) as req:                # Daten herunterladen
        text = req.read().decode()                   # Daten einlesen
        if play is None:                             # Stück gewünscht?
            return json.loads(text)                  # JSON der Korpusmetadaten parsen und zurückgeben
        return text                                  # Text des Stückes zurückgeben


def get_data(corpus):
    """Alle Stücke eines Korpus herunterladen."""
    texts = []                                       # Texte der Stücke
    target = []                                      # Autor*innen der Stücke
    for drama in get_dracor(corpus)["dramas"]:       # alle Stücke durchlaufen
        name = drama["name"]                         # Name des Stücks
        authors = drama["authors"]                   # Autor*innen des Stücks
        if len(authors) == 1:                        # nur Stücke mit einem/r Autor*in
            texts.append(get_dracor(corpus, name))   # Text herunterladen
            target.append(authors[0]["fullname"])    # Autor*in hinzufügen
    return texts, target                             # Texte + Autor*innen als Ergebnis

texts, target = get_data("ger")                      # GerDraCor herunterladen

## Textklassifikation

Die meisten Verfahren benötigen numerische Daten für die Klassifikation. Daher müssen wir die Texte zunächst transformieren. Die folgende Funktion wandelt daher die gegebenen Daten mittels einer entsprechenden Transformationsklasse um und trainiert und evaluiert dann einen [Naive-Bayes-Klassifikator für multinomiale Daten](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB), der typischerweise gut für die Textklassifikation geeignet ist:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

def texteval(X, Y, vec):
    X = vec.fit_transform(X)                                  # Textdaten transformieren
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y) # in Test- und Trainingsdaten aufteilen
    clf = MultinomialNB()                                     # Klassifikator instantiieren
    clf.fit(train_X, train_Y)                                 # Model trainieren
    return clf.score(test_X, test_Y)                          # Model evaluieren

Jetzt können wir untersuchen, welchen Einfluss die Art der Texttransformation auf die Klassifikationsgüte hat. 

### Worthäufigkeiten

Beginnen wir mit der einfachsten Möglichkeit: Jedes Dokument wird durch einen Vektor repräsentiert, der für jedes im Korpus vorkommende Wort angibt, wie häufig es im Dokument auftaucht. Das lässt sich mit des [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) umsetzen:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

for i in range(5):                                            # fünf Durchläufe
    print(texteval(texts, target, CountVectorizer()))

#### häufige Wörter

Nur Wörter, die in *mindestens 30%* der Dokumente auftauchen:

In [ ]:
for i in range(5):
    print(texteval(texts, target, CountVectorizer(min_df=0.3)))

#### seltene Wörter

Nur Wörter, die in *höchstens 30%* der Dokumente auftauchen:

In [ ]:
for i in range(5):
    print(texteval(texts, target, CountVectorizer(max_df=0.3)))

#### häufige Bigramme

Nur Bigramme, die in *mindestens 30%* der Dokumente auftauchen:

In [ ]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=0.3)

for i in range(5):
    print(texteval(texts, target, vec))

#### seltene Bigramme

In [ ]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', max_df=0.3)

for i in range(5):
    print(texteval(texts, target, vec))

#### TF-IDF

Häufige Wörter sind oft nicht besonders aussagekräftig für ein Dokument, weswegen die Worthäufigkeit häufig mit der Anzahl der Dokumente in Beziehung gesetzt wird, in denen ein Wort vorkommt. Ein übliches Maß dafür ist [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf):

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

for i in range(5):
    print(texteval(texts, target, TfidfVectorizer(min_df=0.3)))

### Zeichenhäufigkeiten

Diese Experimente können wir auf Zeichenebene wiederholen, dafür müssen wir den `CountVectorizer` lediglich einen anderen Analyzer verwenden lassen:

In [ ]:
for i in range(5):
    print(texteval(texts, target, CountVectorizer(analyzer='char_wb')))

#### häufige Zeichen

In [ ]:
for i in range(5):
    print(texteval(texts, target, CountVectorizer(analyzer='char_wb', min_df=0.3)))

#### seltene Zeichen

In [ ]:
for i in range(5):
    print(texteval(texts, target, CountVectorizer(analyzer='char_wb', max_df=0.3)))

#### häufige Bigramme

In [ ]:
vec = CountVectorizer(ngram_range=(1, 2), analyzer='char_wb', min_df=0.3)

for i in range(5):
    print(texteval(texts, target, vec))

#### seltene Bigramme

In [ ]:
vec = CountVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_df=0.3)

for i in range(5):
    print(texteval(texts, target, vec))